In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv")
train_df #ignore the warnings

In [ ]:
train_df.isnull().sum()

In [ ]:
test_df = pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv")
test_df

In [ ]:
# dropping the unnecesary columns
train_df.drop(columns=['Unnamed: 0'], inplace=True)
test_df.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
# Identifying ndvi columns
ndvi_cols = [col for col in train_df.columns if col.endswith('_N')]

In [ ]:
ndvi_cols

In [ ]:
# Handling values with median 
'''train_df[ndvi_cols] = train_df[ndvi_cols].fillna(train_df[ndvi_cols].median())
test_df[ndvi_cols] = test_df[ndvi_cols].fillna(train_df[ndvi_cols].median()) 
'''

In [ ]:
# using KNN imputer
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
train_df[ndvi_cols] = imputer.fit_transform(train_df[ndvi_cols])
test_df[ndvi_cols] = imputer.transform(test_df[ndvi_cols])


In [ ]:
# summary stats
train_df['ndvi_mean'] = train_df[ndvi_cols].mean(axis=1)
train_df['ndvi_std'] = train_df[ndvi_cols].std(axis=1)
train_df['ndvi_max'] = train_df[ndvi_cols].max(axis=1)
train_df['ndvi_min'] = train_df[ndvi_cols].min(axis=1)

test_df['ndvi_mean'] = test_df[ndvi_cols].mean(axis=1)
test_df['ndvi_std'] = test_df[ndvi_cols].std(axis=1)
test_df['ndvi_max'] = test_df[ndvi_cols].max(axis=1)
test_df['ndvi_min'] = test_df[ndvi_cols].min(axis=1)


In [ ]:
features = ndvi_cols + ['ndvi_mean', 'ndvi_std', 'ndvi_max', 'ndvi_min']


In [ ]:
features

In [ ]:
# encoding target columns
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train_df['class_encoded'] = le.fit_transform(train_df['class'])


In [ ]:
# Feature Scaling 
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(train_df[features])
X_test = scaler.transform(test_df[features])


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [ ]:
# assigning labels
y_train = train_df['class_encoded']
y_train

In [ ]:
# Training
model = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs')
scores = cross_val_score(model, X_train, y_train, cv=15, scoring='accuracy')
print("Cross-validation accuracy of the model:", scores.mean())


In [ ]:
# fit the model
model.fit(X_train, y_train)


In [ ]:
 # checking if model is fitted or not
print(hasattr(model, "coef_")) 

Prediction!!!!!

In [ ]:

preds = model.predict(X_test)

# convert to original class columns
pred_labels = le.inverse_transform(preds)

# submission file 
submission = pd.DataFrame({
    'ID': test_df['ID'],
    'class': pred_labels
})

submission.to_csv('submission.csv', index=False)
